# Basic VAE implementation

This notebook defines a basic VAE to model the MNIST dataset, using PyTorch.

## VAE Theory

The goal of a VAE is to (implicitly) learn a distribution $p(x),\,x \in X$, with $X$ a vector space, which is as similar as possible to an unknown target distribution $p^{*}(x)$, given a set of samples from this distribution $\mathcal{D}=\{x_1, ..., x_{n}\}$. VAEs make the assumption that $x$ depends on some set of underlying latent variables $z \in Z$, which are distributed according to some known distribution $p(z)$, and model the likelihood $p(x\mid z)$ and an approximation of the posterior distribution $p(z \mid x)$.

To help them learn, VAEs use variational inference. In this approach, instead of computing the posterior $p(z \mid x)$, which is often intractable, one searches for a more tractable distribution $q(z \mid x)$ which approximates the posterior. VAEs assume parameterised models of the likelihood and the approximate posterior, given by $p(x \mid z,\theta)$ and $q(z \mid x, \theta)$, where $\theta$ represent model parameters. To learn  $q(z \mid x, \theta)$, we maximise the Evidence Lower Bound (or ELBO) over the training data, which is given by

\begin{equation}
ELBO = \sum_{i=1}^{n} \mathbb{E}_{q(z_{i} \mid x_{i},\theta)} \left[\log p(x_{i} \mid z_{i},\theta) \right] - D_{\mathrm{KL}}(q(z_{i}\mid x_{i},\theta) \, \| \, p(z_{i}))~,
\end{equation}
assuming the training samples are independently and identically distributed. 

To be able to compute the ELBO, we assume a Gaussian VAE:
\begin{align}
\label{eq:vae_likelihood}
p(x \mid z,\theta)&=\mathcal{N}(x; f(z; \theta_{f}), \sigma^{2} I)
\\
\label{eq:vae_prior}
p(z)&=\mathcal{N}(z; 0, I)
\\
\label{eq:vae_q}
q(z\mid x,\theta)&=\mathcal{N}(z; g(x;\theta_{g}), \mathrm{diag}(h(x;\theta_{h})^{2}))~,
\end{align}
where $f$, $g$ and $h$ are all deep neural networks parameterised by $\theta=\{\theta_{f},\theta_{g},\theta_{h}\}$, $\sigma$ is a hyperparameter. Our optimal network parameters are then given by

\begin{equation}
\label{eq:vae_mle}
\hat \theta_{f}, \hat \theta_{g}, \hat \theta_{h} = argmin_{\theta_{f}, \theta_{g}, \theta_{h}}  \left\{ \sum_{i=1}^{n} \mathbb{E}_{q(z_{i} \mid x_{i}, \theta_{g},\theta_{h})} \left[{\| x_{i}-f(z_{i};\theta_{f}) \|^{2} \over 2 \sigma^{2}}\right] + D_{\mathrm{KL}}(q(z_{i}\mid x_{i},\theta_{g},\theta_{h}) \, \| \, p(z_{i})) \right\}~.
\end{equation}

The KL term above can easily be computed analytically as it involves two normal distributions. When calculating the expectation over $q(z\mid x,\theta)$, which is usually carried out via sampling, a "reparameterisation trick" is used. One notes that sampling $z \sim q(z\mid x,\theta)$ is equivalent to sampling $\epsilon \sim \mathcal{N}(\epsilon;0,I)$ and computing $z=h(x;\theta_{h})\epsilon + g(x;\theta_{g})$, which allows gradient descent methods to be used to find $\hat \theta$.

> **Extension task**: show that the KL divergence term in this case can be analytically derived as: $D_{\mathrm{KL}}(q(z_{i}\mid x_{i},\theta_{g},\theta_{h}) \, \| \, p(z_{i}))= -{1\over 2} \sum^{J}_{j=1} \left[ 1 + \log(h_{j}^{2}) - h_{j}^{2} - g_{j}^{2} \right]$ where $J$ is the dimension of the latent vector.

For more on VAE theory, see: https://arxiv.org/abs/1606.05908 and https://arxiv.org/abs/1907.08956.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

## Load training data

First, we load the MNIST training data (similar to before), and plot some training examples.

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_dataset = datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]))

test_dataset = datasets.MNIST('./data', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]))

In [ ]:
print(len(train_dataset))
print(len(test_dataset))

for i in [0, 101]:
    im,label = train_dataset[i]
    
    plt.imshow(im.numpy()[0])
    plt.title(label)
    plt.colorbar()
    plt.show()

print(im.shape)

## Define network

Next we define the VAE.

> **Task 1**: comple the PyTorch VAE model below by implementing the `reparameterize` method which samples from the approximate posterior using the reparameterisation trick, and implementing the model's `forward` method, which returns the estimated parameters $f$,$g$ and $h$ of the likelihood and approximate posterior given a sample $x$.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)# estimates mu, log-variance of approximate posterior q(z|x)

    def reparameterize(self, mu, logvar):
        
        # TODO: write some code which returns a random sample from the approximate posterior q(z|x) here
        


    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))# estimates mu of likelihood p(x|z)

    def forward(self, x):
        
        # TODO: write some code which returns
        # 1) mu,log-variance of the approximate posterior given x
        # 2) mu of the likelihood given a random sample from the posterior given x
        

        
        return recon, mu, logvar# should return  (likelihood mu, posterior mu, posterior log-variance)
    
model = VAE()

## Loss function

Next, we define the loss function to train the VAE model.

> **Task 2**: define a loss function to train the VAE model, which minimises the expression for the negative ELBO shown above.

In [ ]:
def loss_function(recon, x, mu, logvar):
    
    # TODO: write some code here to compute the negative ELBO shown above.
    
    # The reconstruction term can be approximated with a single sample from the approximate posterior.
    # Use the analytical exppression for the KL term.
    # recon, mu and logvar are the outputs from model.forward(x).
    


## Train

Finally, we train the VAE model.

In [ ]:
from torch import optim

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)# this automatically batches up examples, adding a batch dimension
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

> **Task 3**: write some code to train the VAE using the training data and the loss function defined above.

In [ ]:
def train(epoch):
    model.train()
    
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        
        # TODO: write some code here
        # carry out an optimisation step for each batch
        # after each epoch report the average loss over the training dataset
        



def test(epoch):
    model.eval()
    
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            
            # TODO: write some code here
            # report the average loss over the test dataset
            


for epoch in range(1, 5):
    train(epoch)
    test(epoch)

## Evaluate

In this section we evaluate the performance of the trained VAE.

### Reconstruction quality

> **Task 4**: plot example VAE reconstructions (mean of the likelihood) for some of the images in the test set.

In [ ]:
# TODO: write some code here




### Latent space interpolation

Now that we have learnt a latent representation, we can interpolate smoothly between two digits by linearly interpolating between them in the latent space.

> **Task 5**: show the mean of the likelihood (image reconstruction) for a set latent vectors which are linearly interpolated between two test images in the latent space.

In [ ]:
# TODO: write some code here




### Random sampling of the posterior

We can also generate random samples of $x$ given values of $z$ drawn from its prior distribution (a unit Gaussian).

> **Task 6**: generate random latent vectors from the prior latent distribution and show the resulting likelihood means (image reconstructions).

In [ ]:
# TODO: write some code here




## Extensions

Have a go at these if you have time.

> What happens to the performance when you reduce the dimensionality of the latent space?

> Try using a Beta-VAE (https://openreview.net/forum?id=Sy2fzU9gl) : what happens when the KL term is made much stronger?

> Try using a binary cross-entropy loss instead of a L2 loss in the loss function (e.g. https://github.com/pytorch/examples/blob/master/vae/main.py)

> Try to run clustering in the latent space to label the test data.